In [1]:
from pathlib import Path

import pandas as pd # type: ignore


In [2]:
INPUT_PATH = Path('../../output/cre_loader.xlsx')
df = pd.read_excel(INPUT_PATH)

In [ ]:
df

In [3]:
fdic_groups = {
    '1-4 Fam Construction': ['CNFM'],
    'Construction': ['OTCN','LAND','LNDV','RECN'],
    '1-4 Family': ['REFI','REOE','REJU'],
    'OwnerOcc': ['REOW'],
    'I-CRE': ['RENO','REMU'],
    'C&I': ['CIUS'],
    'Other': ['OTAL','AGPR','REFM']
}

In [4]:
call_code_mapping = {code: group for group, codes in fdic_groups.items() for code in codes}
df['Cleaned Call Code'] = df['fdiccatcd'].map(call_code_mapping)

In [5]:
df

,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,bookbalance,...,propaddr1,propaddr2,propcity,propstate,propzip,aprsvalueamt,aprsdate,proptypdesc,propdesc,Cleaned Call Code
0,150936915,REDBROOK APARTMENTS LLC,Commercial Mortgages,65500000.0,VAR,CML,CM40,ACT,0.067809,29000000.00,...,200-204 WAREHAM ROAD,NaN,PLYMOUTH,MA,21845215.0,117100000.0,2023-05-23,Apartment Building,"200-204 Wareham Road, Plymouth, MA 02184-5215",I-CRE
1,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.063500,26000000.00,...,36 SARATOGA BLVD,NaN,DEVENS,MA,14345217.0,21600000.0,2024-05-08,Warehouse,"36 Saratoga Blvd, Devens, MA 01434",I-CRE
2,150862011,"R3 PROJECT COMPANY, LLC",CML Fixed Construction,20000000.0,FIX,CML,CM07,ACT,0.065000,0.00,...,1562 - 1570 HANCOCK ST,NaN,QUINCY,MA,21695205.0,100000000.0,2022-08-14,Multi Family,"1550 Hancock Street, Quincy, MA 02169",Construction
3,151038843,"POWER 250, LLC",CML ARM Construction,27500000.0,VAR,CML,CM08,ACT,0.070865,2254909.06,...,157 GANO ST,NaN,PROVIDENCE,RI,29063808.0,12800000.0,2023-08-24,Apartment Building,"157 Gano Street, Providence, RI 02906",Construction
4,150809211,29 CENTER STREET LLC,Commercial Mortgages,18000000.0,FIX,CML,CM40,ACT,0.049000,17596225.29,...,29 CENTER ST,NaN,NANTUCKET,MA,25543687.0,70200000.0,2022-06-28,Hotel/Motel,"29 Center Street, Nantucket, MA 02554",I-CRE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1876,151105642,"JEFF-ANTHONY PROPERTIES, INC.",ARC Fixed Loan,525000.0,FIX,CML,CM77,ACT,0.061700,0.00,...,1063 MINERAL SPRING AVE,NaN,PROVIDENCE,RI,2904.0,760000.0,2024-07-11,Commercial - Other,1063 Mineral Spring Ave North Providence RI 02...,I-CRE
1877,151130772,WM3 PROPERTIES LLC,ARC Fixed Loan,1000000.0,FIX,CML,CM77,ACT,0.065500,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,I-CRE
1878,100107381751,EMOND PLUMBING AND HEATING INC,Borrowing Base Line of Credit,2200000.0,VAR,CML,CM06,ACT,0.080000,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Bus Assets w/Accts Receivable,"ALL INVENTORY, EQUIPMENT, ACCOUNTS (INCLUDING ...",C&I
1879,500209311111,ATLANTIC ELEVATOR SOUTH CO INC,Equipment Line of Credit,300000.0,VAR,CML,CM11,ACT,0.075000,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Equipment,EQUIPMENT,C&I


In [6]:
total_cml = df.groupby('Cleaned Call Code')['Net Balance'].sum().reset_index()

In [8]:
TOTAL_CML_OUTPUT_PATH = Path('../../output/total_cml.xlsx')
total_cml.to_excel(TOTAL_CML_OUTPUT_PATH, index=False)

In [25]:
total_cml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Cleaned Call Code  7 non-null      object 
 1   Net Balance        7 non-null      float64
dtypes: float64(1), object(1)
memory usage: 244.0+ bytes


In [26]:
icre = df[df['fdiccatcd'].isin(['RENO','REMU'])].copy()

In [27]:
icre['Net Balance'].sum()

np.float64(625416712.49)

In [28]:
icre['proptypdesc'].unique()

array(['Apartment Building', 'Warehouse', 'Hotel/Motel', 'Multi Family',
       'Office- General', 'Office - Medical', 'Commercial - Other',
       'Shopping Plaza', 'Strip Plaza', 'Mixed Use (Retail/Office)',
       'General Retail', 'Parking Lot', 'Mixed Use (Retail/Residential)',
       'Retail - Big Box Store', 'Industrial',
       'Gas Station and Convenience St', 'Self Storage', nan,
       'Land - Improved', 'Manufacturing', 'UCC - ABA',
       'Office - Professional', 'Restaurant', 'Seafood Processing Plant',
       '1-4 Fam Res - Non Own Occ', 'Solar Farm', 'Autobody/Gas Station',
       'Mixed Use (Office/Residential)', 'All Business Assets',
       'Educational Facilities', 'Indoor Recreational',
       '2 Family Residential - Own Occ', 'Golf Course', 'Car Wash'],
      dtype=object)

In [29]:
prop_groups = {
    'Autobody/Gas Station': ['Autobody/Gas Station','Gas Station and Convenience St','Auto-Truck Repair'],
    'Other': ['Other','Commercial - Other'],
    'Retail': ['Retail - Big Box Store','Shopping Plaza','Strip Plaza','Dry Cleaner/Laundromat','General Retail'],
    'Hospitality': ['Hotel/Motel','Hospitality/Event Space'],
    'Recreation': ['Outdoor Recreation','Indoor Recreational'],
    'Industrial': ['Manufacturing','Warehouse'],
    'Land': ['Land - Unimproved','Land - Improved'],
    'Mixed Use': ['Mixed Use (Retail/Office)','Mixed Use (Retail/Residential)','Mixed Use (Office/Residential)'],
    'Multi Family': ['Apartment Building'],
    'General Office': ['Office - Professional','Office- General'],
    'Medical Office': ['Office - Medical'],
    'Restaurant': ['Restaurant']
}

In [30]:
proptype_mapping = {code: group for group, codes in prop_groups.items() for code in codes}
icre['Cleaned PropType'] = icre['proptypdesc'].map(proptype_mapping).fillna(icre['proptypdesc'])


In [31]:
ICRE_OUTPUT_PATH = Path('../../output/icre.xlsx')
icre.to_excel(ICRE_OUTPUT_PATH, index=False)

In [32]:
construction = df[df['Cleaned Call Code'] == 'Construction'].copy()

In [33]:
construction['Net Balance'].sum()

np.float64(64806269.85)

In [34]:
proptype_mapping = {code: group for group, codes in prop_groups.items() for code in codes}
construction['Cleaned PropType'] = construction['proptypdesc'].map(proptype_mapping).fillna(construction['proptypdesc'])
CONSTRUCTION_OUTPUT_PATH = Path('../../output/construction.xlsx')
construction.to_excel(CONSTRUCTION_OUTPUT_PATH, index=False)